#

In [ ]:
import pyaudio
import os
import struct
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft
import time
from tkinter import TclError
from scipy.signal import freqs, bilinear, freqz

# to display in separate Tk window
%matplotlib tk

# constants
CHUNK = 1024 * 4             # samples per frame
FORMAT = pyaudio.paInt16     # audio format (bytes per sample?)
CHANNELS = 1                 # single channel for microphone
RATE = 44100                 # samples per second

# Analoger Biquad Filter
# Bilinear Transformation basierend auf dem Experiment 4 der
# Biquad Schaltung aus Analog System Lab Kit PRO

R = 1000
C = 100e-9
w0 = 1 / (R * C)
Q = 4.7
fs = 44100

# Nenner
dens = [1, w0/Q, w0**2]
# Tiefpass Zähler
TP_nums = [0, 0, w0**2]
ws_TP, Hs_TP = freqs(TP_nums, dens, worN = 512)
# Hochpass Zähler
HP_nums = [1, 0, 0]
ws_HP, Hs_HP = freqs(HP_nums, dens, worN = 512)
# Bandpass Zähler
BP_nums =  [0, -w0, 0]
ws_BP, Hs_BP = freqs(BP_nums, dens, worN = 512)
# Bandstop Zähler
BS_nums = [1, 0, w0**2]
ws_BS, Hs_BS = freqs(BS_nums, dens, worN = 512)

# Tiefpass
TP_numz, TP_denz = bilinear(TP_nums, dens, fs = fs)
wz_TP, Hz_TP = freqz(TP_numz, TP_denz, worN = 512, fs = fs)
# Hochpass
HP_numz, HP_denz = bilinear(HP_nums, dens, fs = fs)
wz_HP, Hz_HP = freqz(HP_numz, HP_denz, worN = 512, fs = fs)
# Bandpass
BP_numz, BP_denz = bilinear(BP_nums, dens, fs = fs)
wz_BP, Hz_BP = freqz(BP_numz, BP_denz, worN = 512, fs = fs)
# Bandstop
BS_numz, BS_denz = bilinear(BS_nums, dens, fs = fs)
wz_BS, Hz_BS = freqz(BS_numz, BS_denz, worN = 512, fs = fs)

In [ ]:
# create matplotlib figure and axes
fig, (ax1, ax2) = plt.subplots(2, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)       # samples (waveform)
xf = np.linspace(0, RATE, CHUNK)     # frequencies (spectrum)

# create a line object with random data
line, = ax1.plot(x, np.random.rand(CHUNK), '--', lw=1)

# create semilogx line for spectrum
line_fft, = ax2.plot(xf, np.random.rand(CHUNK), '-', lw=1)

# Signal range is -32k to 32k
# limiting amplitude to +/- 4k
AMPLITUDE_LIMIT = 32000

plt.plot(wz_TP, 20*np.log10(abs(Hz_TP)), label = 'Tiefpass')
plt.plot(wz_HP, 20*np.log10(np.maximum(abs(Hz_HP), 1e-10)), label = 'Hochpass')
plt.plot(wz_BP, 20*np.log10(np.maximum(abs(Hz_BP), 1e-10)), label = 'Bandpass')
plt.plot(wz_BS, 20*np.log10(abs(Hz_BS)), label = 'Bandstop')
plt.grid(True)
plt.title('Digitale Biquad Frequenzgänge')
plt.xlabel('Frequenz in [Hz]')
plt.ylabel('Amplitude in [dB]')
plt.xlim([0, 4000])
plt.ylim([-30, 15])
plt.legend(loc='lower right')

# format waveform axes
ax1.set_title('AUDIO WAVEFORM')
ax1.set_xlabel('samples')
ax1.set_ylabel('volume')
ax1.set_ylim(-AMPLITUDE_LIMIT, AMPLITUDE_LIMIT)
ax1.set_xlim(0, 2 * CHUNK)

plt.setp(ax1, xticks=[0, CHUNK, 2 * CHUNK], yticks=[-AMPLITUDE_LIMIT, 0, AMPLITUDE_LIMIT])
plt.grid()
# format spectrum axes
ax2.set_xlim(20, RATE / 2)
ax2.set_ylim(-100, 40)
print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK)    

    data_np = np.frombuffer(data, dtype='h')
    
    line.set_ydata(data_np)
    
    # compute FFT and update line
    yf = fft(data_np)
    line_fft.set_ydata(20*np.log10(np.abs(yf[0:CHUNK])  / (512 * CHUNK)))
    
    # update figure canvas
    try:
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except TclError:
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break